In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder


In [ ]:
# decorators.py
import time
def timing(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f"{func.__name__} executed in {time.time() - start:.2f}s")
        return result
    return wrapper


In [ ]:
# Fichiers d'entrée/sortie
input_path = "../data/raw/train.csv"
output_path = "../data/processed/train_clean.csv"

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text.strip()

def data_generator(file_path, chunk_size=10000):
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunk.dropna(subset=['text', 'label'], inplace=True)
        chunk['text'] = chunk['text'].astype(str).apply(preprocess_text)
        yield chunk

# Fonction pour traiter et sauvegarder les données dans le fichier final
def process_data_stream(file_path, output_path):
    first = True  # Pour vérifier si c'est le premier chunk
    for chunk in data_generator(file_path):
        chunk.to_csv(output_path, mode='a', index=False, header=first)  # Sauvegarde les données
        first = False  # Après le premier chunk, ne plus écrire l'en-tête

gen = data_generator(input_path, chunk_size=10000)
first_chunk = next(gen)
first_chunk.head()  # Affiche les premières lignes du premier chunk


In [ ]:
process_data_stream(input_path, output_path)

In [ ]:
# train.py
import mlflow
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
# from data_loader import data_generator  # ou ton code générateur

# Chargement des données nettoyées
gen = data_generator("../data/processed/train_clean.csv", chunk_size=5000)
X_list, y_list = [], []

for chunk in gen:
    X_list.extend(chunk["text"])
    y_list.extend(chunk["label"])

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
X_vect = vectorizer.fit_transform(X_list)
y = y_list

X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)

# Entraînement + suivi MLflow
with mlflow.start_run():
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    preds = clf.predict(X_test)
    acc = accuracy_score(y_test, preds)

    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(clf, "model")
    mlflow.log_param("model", "LogisticRegression")


In [3]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Création d'un jeu de données factice
data = {'text': ["I love programming", "Python is great", "I hate bugs", "Programming is fun", 
                 "I love solving problems", "Bugs are annoying", "Python is awesome", "Debugging is hard"],
        'label': [1, 1, 0, 1, 1, 0, 1, 0]}  # Classe 1: Positif, Classe 0: Négatif

df = pd.DataFrame(data)

# Séparer les données et les étiquettes
X = df['text']
y = df['label']

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Étape 1 : Prétraitement avec TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Limite à 1000 mots les plus fréquents
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Étape 2 : Entraîner le modèle de régression logistique
model = LogisticRegression()

# Entraînement du modèle
model.fit(X_train_tfidf, y_train)

# Prédictions sur les données de test
y_pred = model.predict(X_test_tfidf)

# Calcul de la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Étape 3 : Suivi du modèle avec MLflow
# Démarrer un suivi avec MLflow
with mlflow.start_run():
    # Log de l'accuracy
    mlflow.log_metric("accuracy", accuracy)
    
    # Log du modèle
    mlflow.sklearn.log_model(model, "model")
    
    # Log des paramètres du modèle (exemple de hyperparamètres)
    mlflow.log_param("max_features", 1000)

    # Log du vecteur TF-IDF
    # mlflow.log_artifact("tfidf_vectorizer.pkl")  # Sauvegarde du vectorizer dans un fichier (si nécessaire)
    
    print("Modèle sauvegardé avec MLflow")


Accuracy: 0.5000
Modèle sauvegardé avec MLflow


c:\Users\user\OneDrive\Desktop\MD3S-FP\Semester 1\Python Av\Workspace python\classification-commentaires\.venv\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\user\OneDrive\Desktop\MD3S-FP\Semester 1\Python Av\Workspace python\classification-commentaires\.venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
import mlflow

mlflow.set_experiment("test_experiment")

with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("accuracy", 0.89)
